In [1]:
import torch
import pandas as pd
import numpy as np
from peft import PeftModel
from models.TimeLLM import Model as TimeLLMModel

/home/user/anaconda3/envs/2hg_time-llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-17 14:17:15,294] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/user/anaconda3/envs/2hg_time-llm/compiler_compat/ld: cannot find -laio: 그런 파일이나 디렉터리가 없습니다
collect2: error: ld returned 1 exit status
/home/user/anaconda3/envs/2hg_time-llm/compiler_compat/ld: cannot find -lcufile: 그런 파일이나 디렉터리가 없습��다
collect2: error: ld returned 1 exit status


In [2]:
# ── 1. Config 정의 ──────────────────────────────────────────────────────
class Config:
    # 태스크 및 시퀀스 길이
    task_name = "long_term_forecast"
    seq_len   = 512
    pred_len  = 96

    # 입력 변수 수 (univariate 시계열이면 1)
    enc_in = 21

    # 내부 차원
    d_model    = 32    # PatchEmbedding 차원
    d_ff       = 32    # ReprogrammingLayer 내부 FF 차원
    llm_dim    = 5120   # LLM 임베딩 차원
    llm_layers = 48    # LLM 레이어 수
    n_heads    = 8

    # 패치 설정
    patch_len = 16
    stride    = 8

    # 드롭아웃
    dropout = 0.1

    # (프롬프트 도메인 사용 안 함)
    prompt_domain = False
    content       = "TimeLLM-Weather"

    # LoRA 설정 (여기서는 False로 두고, 이후 llm_model만 패치)
    lora               = False
    lora_r             = 8
    lora_alpha         = 32
    lora_dropout       = 0.1
    lora_target_modules= ["q_proj","k_proj","v_proj","o_proj"]

    llm_model="QWEN"

configs = Config()
device = "cuda:3" if torch.cuda.is_available() else "cpu"

In [3]:
# ── 2. TimeLLM 모델 인스턴스화 ────────────────────────────────────────
base_model = TimeLLMModel(configs).to(device)
base_model = base_model.to(torch.bfloat16)

Loading checkpoint shards: 100%|██████████| 8/8 [00:03<00:00,  2.06it/s]


In [4]:
# ── 3. LoRA 어댑터만 llm_model에 패치 (체크포인트 경로를 실제로 맞춰주세요) ──
lora_checkpoint = "/home/user/2HG/Time-LLM/checkpoints/long_term_forecast_weather_512_96_TimeLLM_Weather_ftM_sl512_ll48_pl96_dm32_nh8_el2_dl1_df32_fc3_ebtimeF_test_0-TimeLLM-Weather/checkpoint_epoch2_iter9000"
base_model.llm_model = PeftModel.from_pretrained(
    base_model.llm_model,
    lora_checkpoint,
)
model = base_model
model.eval()

Model(
  (llm_model): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 5120)
        (layers): ModuleList(
          (0-47): 48 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=5120, out_features=5120, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
         

In [5]:
# ── 4. 입력 데이터 로드 & 히스토리 준비 ───────────────────────────────
df = pd.read_csv("/home/user/2HG/Time-LLM/dataset/weather/weather_inf2.csv", parse_dates=True, index_col=0)
# (1) multivariate 히스토리: 마지막 seq_len 행, 모든 컬럼
history = df.values[-configs.seq_len:]       # shape (seq_len, n_vars)
x_enc = torch.tensor(history, dtype=torch.float32) \
             .unsqueeze(0)  # → (1, seq_len, n_vars)
x_enc = x_enc.to(device)

# (2) 마킹/디코더 입력은 사용되지 않으므로 더미로 생성
#     shape (1, seq_len, 1) / (1, pred_len, 1) 모두 가능
x_mark_enc = torch.zeros(1, configs.seq_len, 1).to(device)
x_dec      = torch.zeros(1, configs.pred_len, 1).to(device)
x_mark_dec = torch.zeros(1, configs.pred_len, 1).to(device)

In [6]:
# ── 5. 순전파로 예측 수행 ────────────────────────────────────────────
with torch.no_grad():
    # y_pred shape = (batch=1, pred_len, enc_in)
    y_pred = model(x_enc, x_mark_enc, x_dec, x_mark_dec)

In [7]:
# ── 6. 결과 NumPy 배열로 변환 ────────────────────────────────────────
forecast = y_pred.squeeze(0).cpu().numpy()  # (pred_len,)

print("Next 96‑step forecast (shape={}):".format(forecast.shape))
print(forecast)

Next 96‑step forecast (shape=(96, 21)):
[[985.88947   18.878614 293.1527   ... 480.17798   28.947504 413.87958 ]
 [986.3342    19.138489 293.5139   ... 563.99603   29.509834 415.8713  ]
 [986.4221    19.337273 293.63867  ... 627.1677    30.106148 416.59427 ]
 ...
 [985.91986   18.77389  293.01065  ... 466.6192    29.458714 414.2543  ]
 [985.7716    18.735102 292.87674  ... 413.6166    29.244934 412.8206  ]
 [985.3192    18.23087  292.48715  ... 205.30408   28.696547 411.22394 ]]


In [8]:
# ── 7. DataFrame 생성 ───────────────────────────────────────────────────────────
# (1) 원본 df에서 컬럼명 가져오기
orig_df = pd.read_csv("/home/user/2HG/Time-LLM/dataset/weather/weather_inf2.csv", index_col=0, parse_dates=True)
cols    = orig_df.columns.tolist()

# (2) 예측 결과로 DataFrame 만들기
#     만약 시간 인덱스를 그대로 이어가고 싶다면 아래 주석을 참고하세요.
forecast_df = pd.DataFrame(forecast, columns=cols)

# ── (선택) datetime index 이어 붙이기 ───────────────────────────────────────
# 원본 인덱스가 DatetimeIndex이고 freq 정보가 있다면:
last_ts = orig_df.index[-1]
freq    = orig_df.index.freq or (orig_df.index[-1] - orig_df.index[-2])
future_idx = pd.date_range(start=last_ts + freq, periods=forecast.shape[0], freq=freq)
forecast_df.index = future_idx

In [9]:
# ── 8. CSV로 저장 ─────────────────────────────────────────────────────────────
out_path = "weather_forecast.csv"
forecast_df.to_csv(out_path, index=False)  # index=True 로 하면 인덱스도 같이 저장

print(f"▶ Forecast saved to {out_path}")

▶ Forecast saved to weather_forecast.csv
